In [6]:
import os
from dotenv import load_dotenv
from huggingface_hub import InferenceClient

load_dotenv()
client = InferenceClient(api_key=os.getenv("HF_TOKEN"))

In [7]:
for message in client.chat_completion(
	model="microsoft/Phi-3.5-mini-instruct",
	messages=[{"role": "user", "content": "What is the capital of the United States?"}],
	max_tokens=500,
	stream=True,
):
    print(message.choices[0].delta.content, end="")

 The capital of the United States is Washington, D.C. It was established as the national capital on June 11, 1790, and is located on the Potomac River. Washington, D.C. stands for "District of Columbia." Originally named Columbia after Christopher Columbus, it was renamed to honor George Washington after his presidency. The city is not part of any state and is unique in that it is under the exclusive jurisdiction of Congress. Washington, D.C., is home to iconic buildings such as the White House, where the President of the United States resides, and the Capitol Building, which houses the United States Congress. Additionally, numerous national museums, monuments, and memorials, like the Lincoln Memorial and the Smithsonian Institution, enrich the city’s significance as a center of political, cultural, and historical importance in the U.S.

Additionally, Washington, D.C. has its own local government, established by the District of Columbia Home Rule Act in 1973. The heads of both the exec

In [17]:
for message in client.chat_completion(
	model="meta-llama/Llama-3.2-1B-Instruct",
	messages=[{"role": "user", "content": "What is a knowledge graph?"}],
	max_tokens=500,
	stream=True,
):
    print(message.choices[0].delta.content, end="")

A knowledge graph (kg) is a digital representation of a complex system, relationship, or entity, using a graph structure to organize and visualize the relationships between concepts, entities, and their interactions. It's like a massive database that provides a layer of abstraction and insight into the interconnected nature of things.

A typical knowledge graph consists of multiple layers of nodes and edges, where each node represents an instance, concept, or entity, and each edge represents a relationship or connection between them. These edges can be weighted, indicating the strength or significance of the connection.

Knowledge graphs have numerous applications across various fields, including:

1. **Data integration**: KGs help unify data from multiple sources, enabling data integration and real-time data fusion.
2. **Natural Language Processing (NLP)**: KGs are used in language models to represent entities, relationships, and context, enabling more effective text analysis, search,

In [13]:
image_url = "https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg"

for message in client.chat_completion(
	model="meta-llama/Llama-3.2-11B-Vision-Instruct",
	messages=[
		{
			"role": "user",
			"content": [
				{"type": "image_url", "image_url": {"url": image_url}},
				{"type": "text", "text": "What is happening in this image?"},
			],
		}
	],
	max_tokens=500,
	stream=True,
):
	print(message.choices[0].delta.content, end="")

The image shows the Statue of Liberty, a symbol of freedom and democracy, in the foreground. The statue is situated in New York Harbor, with the iconic Manhattan skyline visible in the background.

In [24]:
import dspy
from textwrap import dedent

os.environ["HUGGINGFACE_API_KEY"] = os.environ["HF_TOKEN"]
lm = dspy.LM('huggingface/meta-llama/Llama-3.2-11B-Vision-Instruct')
dspy.configure(lm=lm)

In [31]:
ques = dedent(
    """
    Which of the following is the function if the integral is 8?
    """
)

ans = dedent(
    """
    f(x) = 4x between 0 and 1
    """
)

class ExplainMistake(dspy.Signature):
    """
    Describe how the user would approach this problem and why they answered it incorrectly.
    """
    question = dspy.InputField(desc="The question posed to the student.")
    answer = dspy.InputField(desc="The incorrect answer that the student provided.")
    mistake = dspy.OutputField(desc="The mistake that the student made and why it would lead to a false answer.")

subject = dspy.ChainOfThought(ExplainMistake)
subject(question=ques, answer=ans)

Prediction(
    reasoning='The reasoning behind this question is that the function f(x) = 4x is a linear function, and the integral of a linear function between two points is the average of the function values at those points multiplied by the distance between the points. In this case, the average of f(0) and f(1) is (0 + 4) / 2 = 2, and the distance between 0 and 1 is 1. Therefore, the integral is 2 * 1 = 2, not 8.',
    mistake='The mistake the student made is that they incorrectly calculated the integral of the function f(x) = 4x between 0 and 1. They likely misunderstood the formula for the integral of a linear function or made a calculation error.'
)